# Signal and background models

We now have all the components necessary for a complete signal model. We put them all together, and bin the spectrum according to the JUNO conventions. We also show the spectra of background components, digitized from the JUNO publication。